In [2]:
import numpy as np
import os
import pandas as pd
import cv2
import h5py

from scipy.ndimage import convolve, correlate
from scipy.signal import medfilt2d

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import LeaveOneOut

import matplotlib.pyplot as plt

In [10]:
pars = {'ks1':[],'sigma1':[],'ks2':[],'sigma2':[],'R':[],'r':[],'wc':[],'wR':[],'wr':[]}

for k in range(200):
    pars['ks1'].append(np.random.randint(3,26))
    pars['sigma1'].append(np.random.randint(1,pars['ks1'][-1]+1))
    pars['ks2'].append(np.random.randint(3,26))
    pars['sigma2'].append(np.random.randint(1,pars['ks2'][-1]+1))
    pars['R'].append(np.random.randint(3,28))
    pars['r'].append(np.random.randint(1,pars['R'][-1]))
    pars['wc'].append(np.random.randint(3,26))
    pars['wR'].append(np.random.randint(3,26))
    pars['wr'].append(np.random.randint(3,26))
    
for k in range(len(pars['ks1'])):
    print([pars['ks1'][k],pars['sigma1'][k],pars['ks2'][k],pars['sigma2'][k],
           pars['R'][k],pars['r'][k],pars['wc'][k],pars['wR'][k],pars['wr'][k]])
    print('')

[10, 3, 15, 6, 9, 4, 4, 4, 6]

[7, 1, 23, 4, 10, 8, 4, 6, 19]

[17, 12, 13, 1, 27, 19, 7, 20, 5]

[22, 22, 9, 4, 20, 17, 3, 18, 6]

[11, 10, 17, 12, 6, 5, 4, 10, 24]

[8, 7, 24, 10, 13, 12, 17, 9, 16]

[22, 9, 12, 9, 12, 8, 25, 20, 21]

[21, 9, 9, 2, 27, 8, 16, 18, 7]

[15, 11, 3, 3, 5, 1, 14, 14, 17]

[17, 14, 7, 3, 17, 4, 6, 11, 3]

[18, 4, 22, 13, 11, 3, 10, 6, 16]

[18, 15, 17, 14, 10, 2, 19, 12, 24]

[11, 2, 11, 2, 8, 4, 4, 20, 10]

[23, 15, 10, 2, 15, 10, 11, 21, 11]

[25, 7, 3, 2, 11, 9, 3, 18, 22]

[5, 5, 22, 2, 5, 2, 6, 19, 24]

[23, 13, 22, 16, 25, 3, 22, 6, 18]

[15, 11, 22, 20, 15, 1, 4, 24, 21]

[5, 4, 20, 7, 19, 6, 19, 13, 6]

[13, 2, 5, 1, 18, 13, 24, 3, 17]

[19, 7, 10, 1, 3, 2, 10, 20, 24]

[18, 14, 16, 10, 25, 12, 9, 13, 16]

[11, 7, 21, 11, 21, 12, 14, 12, 23]

[9, 8, 22, 7, 8, 7, 10, 16, 4]

[5, 1, 10, 6, 18, 6, 8, 23, 8]

[5, 2, 24, 19, 25, 18, 21, 12, 10]

[19, 4, 8, 5, 3, 1, 18, 16, 12]

[10, 1, 3, 3, 17, 1, 20, 13, 3]

[22, 13, 16, 16, 23, 10, 3, 13, 21]

[16, 5

In [20]:
pars = []

for k in range(200):
    tmp = {'ks1':0,'sigma1':0,'ks2':0,'sigma2':0,'R':0,'r':0,'wc':0,'wR':0,'wr':0}
    tmp['ks1'] = np.random.randint(1,13)*2+1
    tmp['sigma1'] = np.random.randint(1,tmp['ks1']+1)
    tmp['ks2'] = np.random.randint(1,13)*2+1
    tmp['sigma2'] = np.random.randint(1,tmp['ks2']+1)
    tmp['R'] = np.random.randint(3,28)
    tmp['r'] = np.random.randint(1,tmp['R'])
    tmp['wc'] = np.random.randint(1,8)*2+1
    tmp['wR'] = np.random.randint(1,8)*2+1
    tmp['wr'] = np.random.randint(1,8)*2+1
    pars.append(tmp)

In [21]:
for p in pars:
    print(p)
    print('')

{'ks1': 13, 'sigma1': 3, 'ks2': 25, 'sigma2': 19, 'R': 19, 'r': 4, 'wc': 5, 'wR': 7, 'wr': 5}

{'ks1': 25, 'sigma1': 20, 'ks2': 23, 'sigma2': 9, 'R': 18, 'r': 8, 'wc': 11, 'wR': 11, 'wr': 7}

{'ks1': 23, 'sigma1': 5, 'ks2': 13, 'sigma2': 9, 'R': 11, 'r': 5, 'wc': 13, 'wR': 9, 'wr': 5}

{'ks1': 13, 'sigma1': 10, 'ks2': 19, 'sigma2': 5, 'R': 26, 'r': 16, 'wc': 7, 'wR': 15, 'wr': 11}

{'ks1': 19, 'sigma1': 18, 'ks2': 3, 'sigma2': 2, 'R': 3, 'r': 1, 'wc': 11, 'wR': 7, 'wr': 13}

{'ks1': 15, 'sigma1': 11, 'ks2': 21, 'sigma2': 9, 'R': 15, 'r': 6, 'wc': 3, 'wR': 11, 'wr': 9}

{'ks1': 25, 'sigma1': 3, 'ks2': 19, 'sigma2': 4, 'R': 10, 'r': 7, 'wc': 15, 'wR': 3, 'wr': 13}

{'ks1': 17, 'sigma1': 14, 'ks2': 23, 'sigma2': 13, 'R': 24, 'r': 12, 'wc': 9, 'wR': 15, 'wr': 3}

{'ks1': 3, 'sigma1': 3, 'ks2': 11, 'sigma2': 11, 'R': 10, 'r': 3, 'wc': 15, 'wR': 7, 'wr': 13}

{'ks1': 15, 'sigma1': 2, 'ks2': 21, 'sigma2': 15, 'R': 9, 'r': 5, 'wc': 3, 'wR': 3, 'wr': 9}

{'ks1': 21, 'sigma1': 14, 'ks2': 21, 'si

In [23]:
A = [1,2,3,4,5]
print(A[1:-1])

[2, 3, 4]
